<a href="https://colab.research.google.com/github/andricmitrovic/YOLO-object-detection/blob/main/models/model_resnet50%2BCNN_BLOCK_8%2BFC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
from torchvision import models

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
class Yolov1(nn.Module):
    def __init__(self, **kwargs):
        super(Yolov1, self).__init__()
        self.model = self._create_model(**kwargs)

    def forward(self, x):
        x = self.model(x)
        return x

    def _create_model(self, **kwargs):
        model = self._create_darknet()

        num_ftrs = model.fc.in_features
        fcs_layers = self._create_fcs(num_ftrs, **kwargs)

        model.fc = fcs_layers
        return model

    def _create_darknet(self):
        darknet = models.resnet50(pretrained=True)
        ct = 0
        for child in darknet.children():
            ct += 1
            if ct < 8:
                for param in child.parameters():
                    param.requires_grad = False
            else:
                print(f"Unfrozen layer block {ct}:")
                print(child)
                print()
        return darknet

    def _create_fcs(self, num_ftrs, split_size, num_boxes, num_classes):
        S, B, C = split_size, num_boxes, num_classes

        return nn.Sequential(
          nn.Linear(num_ftrs, 4096),
          nn.LeakyReLU(0.1),
          nn.Linear(4096, S * S * (C + B * 5)),
        )

In [ ]:
def test(S = 7, B = 2, C = 3):
    model = Yolov1(split_size = S, num_boxes = B, num_classes = C)
    x = torch.randn((2, 3, 224, 224))                  # !!! resnet accepets input in this shape (3 x H x W)
    print(model(x).shape)

In [ ]:
# test()

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Unfrozen layer block 8:
Sequential(
  (0): BasicBlock(
    (conv1): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (downsample): Sequential(
      (0): Conv2d(256, 512, kernel_size=(1, 1), stride=(2, 2), bias=False)
      (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (1): BasicBlock(
    (conv1): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2)